In [ ]:
# install stuff, set scientific variables to 999 to avoid scientific notation
if (!require("pacman")) install.packages("pacman")
pacman::p_load(WDI, tidyr, car, lmtest, robustbase, MASS, dplyr, knitr, broom)


In [ ]:
options(scipen = 999)


In [ ]:
# get all of the WDI indicators if indicator file is not already downloaded
indicators <- if (file.exists("indicators.csv")) {
    read.csv("indicators.csv")
} else {
    WDI(indicator = c(
        "SL.EMP.TOTL.SP.ZS", "SE.XPD.TOTL.GD.ZS", "NY.GDP.PCAP.CD",
        "SM.POP.NETM", "SE.TER.CUAT.BA.ZS", "SP.POP.TOTL", "IP.PAT.RESD"
    ), country = "all") %>%
        write.csv("indicators.csv")
}
# rename the columns
wdi_indicators <- rename(
    indicators,
    c(
        "Country Code" = "iso3c", "Total Employment" = "SL.EMP.TOTL.SP.ZS",
        "Education Expenditure % of GDP" = "SE.XPD.TOTL.GD.ZS", "GDP per capita" = "NY.GDP.PCAP.CD",
        "Net Migration" = "SM.POP.NETM", "PCT Tertiary Education" = "SE.TER.CUAT.BA.ZS",
        "Total Population" = "SP.POP.TOTL", "PatentsByResidents" = "IP.PAT.RESD"
    )
)


In [ ]:
gii <- read.csv("./gii_analysis/gii_2013_2020.csv")
# filter out the rows where the indicator is "Global Innovation Index" and the subindicator type is "Score"
gii_score <- gii %>%
    filter(
        Indicator == "Global Innovation Index",
        Subindicator.Type == "Score (0-100)"
    )
gii_score <- gii_score %>%
    gather(year, value, X2013:X2020) %>%
    select(-Indicator, -Indicator.Id, -Country.Name) %>%
    spread(Subindicator.Type, value)
gii_score <- rename(gii_score, c("Score" = "Score (0-100)"))

# filter out the rows where the indicator is "Global Innovation Index" and the subindicator type is "Rank"
gii_rank <- gii %>%
    filter(Indicator == "Global Innovation Index", Subindicator.Type == "Rank")
gii_rank <- gii_rank %>%
    gather(year, value, X2013:X2020) %>%
    select(-Indicator, -Country.Name, -Indicator.Id) %>%
    spread(Subindicator.Type, value)

# merge into one dataframe
gii_rank_score <- merge(gii_score, gii_rank, by = c("Country.ISO3", "year"))
# remove the X from the year column and convert to integer
gii_rank_score$year <- as.integer(gsub("X", "", gii_rank_score$year))


In [ ]:
# merge gii_rank_score with wdi_indicators
gii_wdi <- gii_rank_score %>% right_join(wdi_indicators, by = c("Country.ISO3" = "Country Code", "year" = "year"))
# rename the columns
gii_wdi <- gii_wdi %>%
    rename(c(
        "CountryCode" = "Country.ISO3", "Year" = "year", "Edu" = "PCT Tertiary Education",
        "Mig" = "Net Migration", "EduExp" = "Education Expenditure % of GDP", "TotEmp" = "Total Employment",
        "TotPop" = "Total Population", "Pat" = "PatentsByResidents", "GdpPerCap" = "GDP per capita"
    )) %>%
    select("Year", "CountryCode", "Score", "Rank", "Edu", "Mig", "EduExp", "TotEmp", "TotPop", "Pat", "GdpPerCap") %>%
    arrange(CountryCode, Year) %>%
    filter(!(CountryCode == ""))

# write out gii_wdi to csv if it doesn't already exist
if (!file.exists("./gii_analysis/gii_wdi.csv")) write.csv(gii_wdi, "./gii_analysis/gii_wdi.csv")

# lag everything.
gii_wdi <- gii_wdi %>%
    group_by(CountryCode) %>%
    mutate(lag.Mig01 = lag(Mig, n = 1, default = NA)) %>%
    mutate(lag.Mig05 = lag(Mig, n = 5, default = NA)) %>%
    mutate(lag.Mig10 = lag(Mig, n = 10, default = NA)) %>%
    mutate(lag.Mig20 = lag(Mig, n = 20, default = NA)) %>%
    mutate(lag.Edu01 = lag(Edu, n = 1, default = NA)) %>%
    mutate(lag.Edu05 = lag(Edu, n = 5, default = NA)) %>%
    mutate(lag.Edu10 = lag(Edu, n = 10, default = NA)) %>%
    mutate(lag.Edu20 = lag(Edu, n = 20, default = NA)) %>%
    mutate(lag.EduExp01 = lag(EduExp, n = 1, default = NA)) %>%
    mutate(lag.EduExp05 = lag(EduExp, n = 5, default = NA)) %>%
    mutate(lag.EduExp10 = lag(EduExp, n = 10, default = NA)) %>%
    mutate(lag.EduExp20 = lag(EduExp, n = 20, default = NA)) %>%
    mutate(lag.TotEmp01 = lag(TotEmp, n = 1, default = NA)) %>%
    mutate(lag.TotEmp05 = lag(TotEmp, n = 5, default = NA)) %>%
    mutate(lag.TotEmp10 = lag(TotEmp, n = 10, default = NA)) %>%
    mutate(lag.TotEmp20 = lag(TotEmp, n = 20, default = NA)) %>%
    mutate(lag.TotPop01 = lag(TotPop, n = 1, default = NA)) %>%
    mutate(lag.TotPop05 = lag(TotPop, n = 5, default = NA)) %>%
    mutate(lag.TotPop10 = lag(TotPop, n = 10, default = NA)) %>%
    mutate(lag.TotPop20 = lag(TotPop, n = 20, default = NA))
head(gii_wdi)

# write out gii_wdi_lagged to csv if it doesn't already exist
if (!file.exists("./gii_analysis/gii_wdi_lagged.csv")) write.csv(gii_wdi, "./gii_analysis/gii_wdi_lagged.csv")


In [ ]:
lag1_Edu_EduExp_Mig_TotEmp <- lm(Score ~ lag.Mig01 + lag.Edu01 + lag.TotEmp01 + lag.EduExp01, data = gii_wdi)
vif(lag1_Edu_EduExp_Mig_TotEmp)
bptest(lag1_Edu_EduExp_Mig_TotEmp)
nobs(lag1_Edu_EduExp_Mig_TotEmp)
summary(lag1_Edu_EduExp_Mig_TotEmp)


In [ ]:
lag5_Edu_EduExp_Mig_TotEmp <- rlm(Score ~ lag.Mig05 + lag.Edu05 + lag.TotEmp05 + lag.EduExp05, data = gii_wdi)
vif(lag5_Edu_EduExp_Mig_TotEmp)
bptest(lag5_Edu_EduExp_Mig_TotEmp)
nobs(lag5_Edu_EduExp_Mig_TotEmp)
summary(lag5_Edu_EduExp_Mig_TotEmp)


In [ ]:
model4 <- rlm(Score ~ Mig + Edu + Mig:Edu, data = gii_wdi)
nobs(model4)
summary(model4)


In [ ]:
model5 <- lm(Score ~ Mig + EduExp + Mig:EduExp, data = gii_wdi)
nobs(model5)
summary(model5)


In [ ]:
lag1_Edu_EduExp_Mig <- rlm(Score ~ lag.Edu01 + lag.EduExp01 + lag.Mig01, data = gii_wdi)
vif(lag1_Edu_EduExp_Mig)
bptest(lag1_Edu_EduExp_Mig)
nobs(lag1_Edu_EduExp_Mig)
summary(lag1_Edu_EduExp_Mig)


In [ ]:
lag5_Edu_EduExp_Mig <- rlm(Score ~ lag.Edu05 + lag.EduExp05 + lag.Mig05, data = gii_wdi)
vif(lag5_Edu_EduExp_Mig)
Anova(lag5_Edu_EduExp_Mig)
bptest(lag5_Edu_EduExp_Mig)
nobs(lag5_Edu_EduExp_Mig)
summary(lag5_Edu_EduExp_Mig)


There is a ton of heteroskedasticity so we have to log the dependent variable in an atttempt to defeat such


In [ ]:
# # log score
# log_lag5_Edu_EduExp_Mig <- rlm(log(Pat) ~ Pat + Mig, data = gii_wdi)
# nobs(log_lag5_Edu_EduExp_Mig)
# Anova(log_lag5_Edu_EduExp_Mig)
# car::vif(log_lag5_Edu_EduExp_Mig)
# lmtest::bptest(log_lag5_Edu_EduExp_Mig)
# summary(log_lag5_Edu_EduExp_Mig)
# coeftest(log_lag5_Edu_EduExp_Mig)
# plot(log_lag5_Edu_EduExp_Mig)
pat_lagMig <- rlm(log(Score) ~ Edu + lag.Mig20, data = gii_wdi)
nobs(pat_lagMig)
vif(pat_lagMig)
Anova(pat_lagMig)
coeftest(pat_lagMig)


In [ ]:
rl <- rlm(log(Score) ~ lag.Mig05, data = gii_wdi)
nobs(rl)
summary(rl)
plot(rl)


In [ ]:
log_everything_score_Mig05 <- rlm(log(Score) ~ log(Edu) + log(EduExp) + log(lag.Mig05) + log(TotEmp) + log(TotPop), data = gii_wdi)
nobs(log_everything_score_Mig05)
vif(log_everything_score_Mig05)
Anova(log_everything_score_Mig05)
bptest(log_everything_score_Mig05)
summary(log_everything_score_Mig05)
coeftest(log_everything_score_Mig05)
plot(log_everything_score_Mig05)

In [ ]:
log_everything_score_Mig01 <- rlm(log(Score) ~ log(Edu) + log(EduExp) + log(lag.Mig01) + log(TotEmp) + log(TotPop), data = gii_wdi)
nobs(log_everything_score_Mig01)
vif(log_everything_score_Mig01)
Anova(log_everything_score_Mig01)
bptest(log_everything_score_Mig01)
summary(log_everything_score_Mig01)
coeftest(log_everything_score_Mig01)
plot(log_everything_score_Mig01)

In [ ]:
log_everything_score_Mig10 <- rlm(log(Score) ~ log(Edu) + log(EduExp) + log(lag.Mig10) + log(TotEmp) + log(TotPop), data = gii_wdi)
nobs(log_everything_score_Mig10)
vif(log_everything_score_Mig10)
Anova(log_everything_score_Mig10)
bptest(log_everything_score_Mig10)
summary(log_everything_score_Mig10)
coeftest(log_everything_score_Mig10)
plot(log_everything_score_Mig10)